# Startup Lending Model

Insert a description of the model

- [**Setup**](#Setup): Runs any imports and other setup
- [**Inputs**](#Inputs): Defines the inputs for the model
- [**Model**](#Model)
- [**Output**](#Output)


## Setup

Setup for the later calculations are here. The necessary packages are imported.

In [362]:
from dataclasses import dataclass
import numpy_financial as npf
import pandas as pd
import matplotlib.pyplot as plt
import random
import itertools
import matplotlib.colors as mcolors

## Inputs

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created.

In [363]:
@dataclass
class ModelInputs:
    price_machine: float = 1_000_000
    loan_life: tuple = (5, 10, 20)
    initial_default: tuple = (0.1, 0.2, 0.3)
    default_decay: float = 0.9
    final_default: float = 0.4
    recovery_rate: float = 0.4
    interest: tuple = (0.3, 0.35, 0.4)
    num_iterations: int = 1000
    case_names: tuple = ('Fulfillment', 'Default')
        
model_data = ModelInputs()

## Model

In [364]:
def default_risk_at_year(data: ModelInputs, year):
    """
    Gets the default risk at a given year with its decreases overtime as the business matures and a high probability of default in the final year.
    """
    if year < data.loan_life:
        if year == 1:
            default_probability = data.initial_default
        else:
            default_probability = default_risk_at_year (data, year-1) * data.default_decay
    else:
        default_probability = data.final_default
    return default_probability

### Internal Randomness
First, a function to determine the default chance in the year we are in.
- 0: Fulfillment
- 1: Default

As this aligns with how the inputs are defined, the function will return the case number.

In [365]:
def get_the_payment_case (data: ModelInputs, year):
    """
    0 corresponsds to the fulfillment and 1 to the default.
    """
    case_num = random.choices([0,1], weights = [1- default_risk_at_year(data, year), default_risk_at_year(data, year)])[0]
    return case_num

In [366]:
def loan_repayment_at_year(data: ModelInputs, year):
    """
    Gets the loan repayment at a given year from the start of the model based on machine price and interest rate.
    """
    if year < data.loan_life:
        # For n-1 years, the business is responsible for paying the interst.
        CF_at_year = data.price_machine * data.interest
    else:
        # In the final period, both machine price and interest will be paid.
        CF_at_year = data.price_machine * (1 + data.interest)
    return CF_at_year

In [367]:
def loan_assessment (data: ModelInputs, print_output = True):
    """
    The main model function which the internal rate of return based on the model inputs,
    calling the other functions to determine the loan repayment and default probability.
    """
    year = 0
    CF = [-data.price_machine]

    if print_output:
        print ('Loan Schedule:')
        print('Interest rate: {data.interest}, Loan life: {data.loan_life}, Initial default probability: {data.initial_default}')
    while year < data.loan_life :
        year +=1
        default_risk = default_risk_at_year(data, year)
        case = get_the_payment_case(data, year)
        case_type = data.case_names[case]
        # Handling the CF during the loan repayment schedule 
        if case == 0:
            loan_repayment = loan_repayment_at_year(data, year)
            CF.append(loan_repayment)
            if print_output:
                print(f'The loan repayment at year {year} ({case_type}) is ${loan_repayment:,.0f}.') 
        else:
            recovery_amount = data.recovery_rate * data.price_machine
            CF += [0, 0, recovery_amount]
            if print_output:
                print(f'The loan is in {case_type} at year {year}.')
                print(f'There are no interests collected for year {year} and {year + 1}.')
                print(f'The recovery amount collected at year {year + 2} is ${recovery_amount:,.0f}.')
            break

    irr = npf.irr(CF)
    if print_output:
        print(f'\nIRR: The effective interest rate at which the loan payments are valued is: {irr * 100:,.2f}%.')
    return irr

In [368]:
def loan_assessment_iteration_df(data: ModelInputs):
    """
    Runs the model repeatedly based on the number of iterations and puts the resulting
    IRR in a DataFrame
    """
    all_irrs = []
    interest_rate = []
    loan_life = []
    initial_default = []
    
    for i in range(data.num_iterations):
        irr = loan_assessment(data, print_output=False)
        all_irrs.append(irr)
        interest_rate.append(data.interest)
        loan_life.append(data.loan_life)
        initial_default.append(data.initial_default)

    irr_df = pd.DataFrame()
    irr_df['Interest Rate'] = [f"{rate * 100:.0f}%" for rate in interest_rate]
    irr_df['Loan Life'] = loan_life
    irr_df['Initial Default Probability'] = [f"{prob * 100:.0f}%" for prob in initial_default]
    irr_df['IRR'] = [f"{rate * 100:.2f}%" for rate in all_irrs]
    return irr_df

In [369]:
def loan_assessment_summary_df(data):
    combinations = tuple(itertools.product(data.interest, data.loan_life, data.initial_default))
    results_df = pd.DataFrame()

    for comb in combinations:
        interest, loan_life, initial_default = comb
        data = ModelInputs(interest = interest, loan_life = loan_life, initial_default = initial_default)
        irr_df= loan_assessment_iteration_df(data)
        results_df = pd.concat([results_df, irr_df], ignore_index=True)
    
    return results_df

## Output

In [372]:
irr_df = loan_assessment_summary_df(model_data)

## Further Analysis

In [371]:
def summarize_irr_df(irr_df):
    """
    Summarizes the DataFrame which contains IRR from multiple runs
    of the model
    """
    avg_irr = irr_df['IRR'].mean()
    std_irr = irr_df['IRR'].std()
    max_irr = irr_df['IRR'].max()
    min_irr = irr_df['IRR'].min()

    print(
        f'The average IRR is {avg_irr:.0f} across the run inputs, with a standard deviation '
        f'of {std_irr:.1f}, max of {max_irr:.0f}, and min of {min_irr:.0f}.'
    )